# <font color='darkblue'>Modul 3: Übungsmaterial zu Datenverständnis und Datenvorbereitung - Welcher Diamant ist wie wertvoll?</font> &nbsp; <font size='6'>&#128142;</font>

<hr style="border:1px solid gray"> </hr>

In diesem Notebook wird mit einem Datensatz zu Diamanten noch einmal das Bereinigen und Darstellen von Daten geübt. Außerdem werden die im Video thematisierten Skalierungsverfahren praktisch umgesetzt.

## <font color='darkblue'>Inhalt</font>

1. [Datensatz einlesen und erforschen](#kap1)  
    
    
2. [Datensatz vorbereiten und visualisieren](#kap2)     


3. [Skalierung der Daten](#kap3)  
   3.1 [Standardisierung der Daten](#kap31)  
   3.2 [Normalisierung der Daten](#kap32)  
   
   
4. [Fazit](#kap4)

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>1. Datensatz einlesen und erforschen</font> <a name="kap1"></a>

Zunächst einmal wird die Pandas Bibliothek und Pyplot eingebunden. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Anschließend wird der Datensatz eingelesen. Es handelt sich hierbei um einen <a href="https://data.world/nahrin/diamonds">Datensatz</a>, der verschiedene Merkmale zu Diamanten und deren Preis enthält.

In [ ]:
df = pd.read_csv('M3_Uebung_Diamanten.csv', index_col=0)
df.info()

<table align='left'>
    <thead>
        <tr>
          <th style="text-align:left">Spaltenname</th>
          <th style="text-align:left">Bedeutung</th>
          <th style="text-align:left">Weitere Informationen</th>
        </tr>
    </thead>
    <tr>
    <td style="text-align:left">price</td>
    <td style="text-align:left">Preis</td>
    <td style="text-align:left">[USD]</td>
    </tr>
    <tr>
    <td style="text-align:left">carat</td>
    <td style="text-align:left">Gewicht</td>
    <td style="text-align:left">[Kt] bzw. [ct]</td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">cut</td>
    <td style="text-align:left; vertical-align: top">Qualität des Schliffs</td>
    <td style="text-align:left; vertical-align: top">Fair, Good, Very Good, Premium, Ideal</td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">color</td>
    <td style="text-align:left; vertical-align: top">Farbe</td>
    <td style="text-align:left; vertical-align: top">D, E, F, G, H, I, J</td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">clarity</td>
    <td style="text-align:left; vertical-align: top">Reinheit</td>
    <td style="text-align:left; vertical-align: top">I1, SI2, SI1, VS2, VS1, VVS2, VVS1, IF</td>
    </tr>
    <tr>
    <td style="text-align:left">x</td>
    <td style="text-align:left">Länge</td>
    <td style="text-align:left">[mm]</td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">y</td>
    <td style="text-align:left; vertical-align: top">Breite</td>
    <td style="text-align:left; vertical-align: top">[mm]</td>
    </tr>
    <tr>
    <td style="text-align:left">z</td>
    <td style="text-align:left">Tiefe</td>
    <td style="text-align:left">[mm]</td>
    </tr>
    <tr>
    <td style="text-align:left; vertical-align: top">depth</td>
    <td style="text-align:left; vertical-align: top">Gesamttiefe in Prozent: z / mean(x, y) = 2 * z / (x + y)</td>
    <td style="text-align:left; vertical-align: top">[%]</td>
    </tr>
    <tr>
    <td style="text-align:left">table</td>
    <td style="text-align:left">Breite der Oberkante des Diamanten im Verhältnis zur breitesten Stelle</td>
    <td style="text-align:left">[%]</td>
    </tr>
</table>

In diesem Datensatz sind 53.940 Datenpunkte enthalten. Und die erste Ansicht zeigt, dass alle Felder gefüllt sind. Das erspart Arbeit bei der Datenbereinigung.
Drei Merkmale haben den Typ 'object', dies sind `cut`, `color` und `clarity`. 


In [ ]:
df.describe()

In [ ]:
df.hist(bins=50, figsize=(12,9)); 

Viele Merkmale enthalten Ausreißer. Dies zeit sich beim Histogramm an einer großen Spannweite auf der x-Achse, die kaum ausgefüllt ist. 

Da die Merkmale `cut`, `color` und `clarity` vom Typ `object` sind und deswegen keine numerischen Ausprägungen haben, sind sie nicht in der Ausgabe von `describe` oder den Histogrammen abgebildet.

Um die tatsächlichen Ausprägungen zu sehen, sollen die ersten 5 Einträge des Datensatzes ausgegeben werden.

In [ ]:
df.head()

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>2. Datensatz vorbereiten und visualisieren</font> <a name="kap2"></a>

Damit die Werte auch visualisiert werden können, soll zunächst eine Umwandlung vorgenommen werden. Um zu ermitteln, welche Ausprägungen das jeweilige Merkmal hat, wird die Funktion `unique()` angewendet, die diese ausgibt:

In [ ]:
pd.unique(df['cut'])

In [ ]:
pd.unique(df['color'])

In [ ]:
pd.unique(df['clarity'])

Mittels Recherche kann ermittelt werden, dass jeweils die folgende Rangfolge gilt:
- Qualität des Schliffs: Fair (schlechteste), Good, Very Good, Premium, Ideal (beste)  
- Reinheit: I1 (schlechteste), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (beste)

Bei diesen Merkmalen liegt also eine Reihenfolge vor. Sie soll also von links nach rechts aufsteigend (Fair &rarr; 0, Good &rarr; 1,...) zahlencodiert werden.

Die Farben haben keine Reihenfolge. Sie sollen alphabetisch absteigend codiert werden, also von D (6) bis J (0). Es ist möglich, dass beispielsweise durch Seltenheit bestimmter Farben ein Einfluss der Farbe auf den Preis dennoch erkennbar ist. Aber eine Rechnung mit diesem Merkmal sollte in jedem Falle inhaltlich hinterfragt werden!

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Wandeln Sie mit Hilfe des Beispiels und der Umwandlung von `cut` die Merkmale `color` und `clarity` in Zahlen-codierte Merkmale um. 
    
</div>

<div class="alert alert-block alert-info">
&#128204; <b>Anmerkung:</b>  
    
Mithilfe der Methode `map()` von Pandas lassen sich Werte in einer Spalte durch andere Werte ersetzen.

Das folgende Beispiel wandelt die Einträge des Merkmals `wetter` Dataframes `df` um. Alle Einträge von `Regen` werden durch den Wert 0, `Bewölkt` durch 1 und `Sonne` durch 2 ersetzt.

```python
df['wetter'] = df['wetter'].map({'Regen': 0, 'Bewölkt': 1, 'Sonne': 2})

```

So entsteht eine Zahlcodierung und es kann außerdem eine Wertung/Rangfolge entstehen: `Regen` < `Bewölkt` < `Sonne`.
    


In [ ]:
# Beispiel für das Merkmal cut
df['cut'] = df['cut'].map({'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4})

In [ ]:
# Konvertieren Sie hier die Merkmale color und clarity

Hat die Umwandlung funktioniert, sollte die folgende Ausgabe der Histogramme nun auch die Markmale `cut`, `color` und `clarity` enthalten. 

In [ ]:
df.hist(figsize=(12,9)) 
plt.show()

Hier ist zu sehen, dass die Merkmalsausprägungen sehr verschiedene Größenordnungen besitzen (z.B. von 0 bis 20000 bei `price` und nur von 0 bis 5 bei `carat`). Dies sorgt dafür, dass vor einer Modellierung eine Skalierung der Daten notwendig ist. Bevor diese Skalierung vorgenommen wird, wird eine letzte etwas aufwändigere Visualisierung vorgenommen. 

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Ergänzen Sie die Variablen, sodass die folgende Zelle einen Scatterplot Karat gegen Preis, eingefärbt nach Reinheit, ausgibt. 
    
</div>

In [ ]:
scatter = plt.scatter(#, #, c=#,marker='o') 
plt.title('Karat, Preis und Reinheit der Diamanten') 
plt.xlabel('Karat') 
plt.ylabel('Preis') 
plt.legend(*scatter.legend_elements())
plt.show;

Das Diagramm zeigt gut erkennbar Farbstufen an. Es gilt, dass bei gleichem Gewicht reinere Diamanten einen höheren Preis erzielen. Die nachfolgende Skalierung wird anhand der in dieser Grafik visualisierten Daten vorgenommen, dafür wird der Datensatz auf einer Kopie auf diese Daten eingeschränkt. 

In [ ]:
df_copy = df[['carat','price','clarity']]

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>3. Skalierung der Daten</font> <a name="kap3"></a>

Wie bereits oben angekündigt, müssen die Daten vor der Modellierung umskaliert werden. Da die Skalierung der Daten stets nach der Trennung der Trainings- und Testdaten vorgenommen wird, muss diese Aufteilung zuerst vorgenommen werden. 

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Führen Sie einen train-test-split durch, bei dem 70% der Daten der Trainingsmenge und 30% der Testmenge zugeordnet wird . Dabei soll das Merkmal `clarity` anteilig gleich in den beiden Mengen vorhanden sein. Außerdem soll auch bei mehreren Durchläufen die gleiche Aufteilung getroffen werden. 
</div>

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_copy, , , ) 

## <font color='darkblue'>3.1 Standardisierung der Daten</font> <a name="kap31"></a>

Zunächst wird die Standardisierung mit dem `StandardScaler` genutzt. Dieser transformiert die Daten so, dass sie schließlich einheitlich skaliert sind, und dass der Mittelwert der Merkmale Null und die Standardabweichung 1 wird.

**Exkurs - StandardScaler (für mathematisch Interessierte):**  

Für eine Spalte mit Werten &ensp; $x_{1},...,x_n$ &ensp; berechnet der StandardScaler zunächst

&ensp;  den Mittelwert &ensp; $m := \frac{1}{n}\sum_{i=1}^nx_i$  
&ensp;  und die Standardabweichung &ensp; $s := \sqrt{\frac{1}{n}\sum_{i=1}^n(x_i-m)^2}$   
&ensp;  und skaliert die einzelnen Werte in der Spalte dann zu neuen Werten &ensp; $z_i := \frac{x_i-m}{s}$.  
    
Diese Spalte mit den neuen Werten $z_1,...,z_n$ hat dann Mittelwert $0$ und Standardabweichung $1$.

Die Skalierung soll nun nur auf die Merkmale `carat` und `price` angewendet werden, damit das Merkmal `clarity` ein Integer bleibt (so kann er für die Farbcodierung eingesetzt werden).   

In [ ]:
from sklearn.preprocessing import StandardScaler

# Nur die Merkmale werden standardisiert, daher wird eine Liste erstellt, die nur die Merkmale ohne das Zielmerkmal enthält
features = list(df_copy.drop(['clarity'], axis=1).columns)

# Skalierung wird an train_set angepasst
scaler = StandardScaler()
scaler.fit(train[features])

# Skalierung der Trainingsdaten
train_features_scaled = pd.DataFrame(scaler.transform(train[features]), columns=features, index=train.index)
# Zusammenfügen der skalierten Merkmale und das Zielmerkmal (nicht skaliert) zu einem Dataframe
train_set_scaled = pd.concat([train_features_scaled, train['clarity']], axis=1)

# Skalierung der Testdaten
test_features_scaled = pd.DataFrame(scaler.transform(test[features]), columns=features, index=test.index)
# Zusammenfügen der skalierten Merkmale und das Zielmerkmal (nicht skaliert) zu einem Dataframe
test_set_scaled = pd.concat([test_features_scaled, test['clarity']], axis=1)

train_set_scaled.head()

Die Ausgabe zeigt bereits, dass sich an den skalierten Merkmalen etwas verändert hat. Nun wird die oben visualisierte Grafik noch einmal für die Trainingsdaten und die skalierten Trainingsdaten ausgegeben (Hier Überprüfung: Stimmt die Syntax von Ihrem Plot?): 

In [ ]:
scatter = plt.scatter(train['carat'], train['price'], c=train['clarity'],marker='o')#löschen
plt.title('Karat, Preis und Reinheit der Diamanten') 
plt.xlabel('Karat') 
plt.ylabel('Preis') 
plt.legend(*scatter.legend_elements())
plt.show;

In [ ]:
scatter = plt.scatter(train_set_scaled['carat'], train_set_scaled['price'], c=train_set_scaled['clarity'],marker='o')#löschen
plt.title('Karat, Preis und Reinheit der Diamanten') 
plt.xlabel('Karat') 
plt.ylabel('Preis') 
plt.legend(*scatter.legend_elements())
plt.show;

Auch die Grafik zeigt: Die Verteilung der Daten ist gleich geblieben, lediglich die Achsen haben sich verändert. 

Es ist zu beachten, dass nun die ursprünglichen Maßeinheiten keinen Sinn mehr ergeben und eine Transformation durchgeführt wurde, die logisch gesehen keinen Sinn hat. Hier werden die Daten jedoch nicht aus der logischen Perspektive, sondern als Data Mining Projekt betrachtet, sodass hier alles erlaubt ist, was für die Modellierung die besten Ergebnisse liefert.

Ein Blick auf die Statistiken der Daten zeigt, dass nun tatsächlich die Mittelwerte (mean) der transformierten Merkmale ungefähr 0 und die Standardabweichungen (std) ungefähr 1 sind:

In [ ]:
train_set_scaled.describe()

## <font color='darkblue'>3.2 Normalisierung der Daten</font> <a name="kap32"></a>

Eine Alternative zur Standardisierung ist die Normalisierung der Daten, beispielsweise mit dem MinMaxScaler. Dieser transformiert die Daten so, dass alle Ausprägungen danach zwischen 0 und 1 liegen. 

**Exkurs - MinMaxScaler (für mathematisch Interessierte):**  

Für eine Spalte mit Werten &ensp; $x_{1},...,x_n$ &ensp; berechnet der MinMaxScaler zunächst

&ensp;  das Maximum $x_{max} := \max\{x_1,\cdots,x_n\}$   
&ensp;  und das Minimum $x_{min} := \min\{x_1,\cdots,x_n\}$   
&ensp;  und skaliert dann zu den neuen Werten $z_i := \frac{x_i-x_{min}}{x_{max}-x_{min}}$.    
    
Die neuen Werte liegen alle zwischen 0 und 1, und die vorherigen Minima sind nun 0 und die vorherigen Maxima sind jetzt 1.

Die Normalisierung soll nun nur auf die Merkmale `carat` und `price` angewendet werden, damit das Merkmal `clarity` ein Integer bleibt (so kann er für die Farbcodierung eingesetzt werden).  

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Nur die Merkmale werden standardisiert, daher wird eine Liste erstellt, die nur die Merkmale ohne das Zielmerkmal enthält
features = list(df_copy.drop(['clarity'], axis=1).columns)

# Skalierung wird an train_set angepasst
scaler = MinMaxScaler()
scaler.fit(train[features])

# Skalierung der Trainingsdaten
train_features_scaled = pd.DataFrame(scaler.transform(train[features]), columns=features, index=train.index)
# Zusammenfügen der skalierten Merkmale und das Zielmerkmal (nicht skaliert) zu einem Dataframe
train_set_scaled = pd.concat([train_features_scaled, train['clarity']], axis=1)

# Skalierung der Testdaten
test_features_scaled = pd.DataFrame(scaler.transform(test[features]), columns=features, index=test.index)
# Zusammenfügen der skalierten Merkmale und das Zielmerkmal (nicht skaliert) zu einem Dataframe
test_set_scaled = pd.concat([test_features_scaled, test['clarity']], axis=1)

train_set_scaled.head()

Die Ausgabe zeigt bereits, dass sich an den skalierten Merkmalen etwas verändert hat. Es soll erneut durch den Plot überprüft werden:

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Legen Sie einen Scatterplot Karat gegen Preis, eingefärbt nach Reinheit, einmal für die Trainingsdaten und einmal für die normierten Trainingsdaten an.
    
</div>

In [ ]:
# Plot für die ursprünglichen Trainingsdaten

In [ ]:
# Plot für die normalisierten Trainingsdaten

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>4. Fazit</font> <a name="kap4"></a>

Dieses Notebook hat
- einige Schritte des Datenverständnisses und der Datenvorbereitung wiederholt. 
- die Umformatierung nicht Zahlen-codierter Merkmale in Zahlen-codierte Merkmale gezeigt.
- die Umsetzung von Skalierungsverfahren in Python erläutert. 